# DDI InfluxDB Timeseries

### Imports

In [1]:
# Imports
import os 
import pandas as pd
from datetime import datetime, timezone
from dotenv import load_dotenv
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
from libs.fetch_data import fetch_data

### Load of env Variables

In [2]:
# Constants
load_dotenv(dotenv_path=".env")

# URLs und Token laden wir weiter aus der .env (das funktioniert ja)
URL   = os.getenv("INFLUX_URL")
ORG   = os.getenv("INFLUX_ORG", "basel-lab")
TOKEN = os.getenv("INFLUX_TOKEN")
PARKING_URL = os.getenv("PARKING_URL")
RAIN_URL = os.getenv("RAIN_URL")
ALRDY_INSTALLED = os.getenv("ALRDY_INSTALLED", 'False').lower() in ('true', '1', 't') 

# WICHTIG: Hier keine getenv() Funktion nutzen, sondern den Namen direkt zuweisen!
BUCKET = "parking_data_final"

print(f"Der Bucket ist gesetzt auf: '{BUCKET}'")

Der Bucket ist gesetzt auf: 'parking_data_final'


### Fetch of Data

In [3]:
if not ALRDY_INSTALLED:
    df_parking, df_rain = fetch_data(PARKING_URL, RAIN_URL)
    df_parking.to_csv("../data/parking.csv")
    df_rain.to_csv("../data/rain.csv")

### Data analysis for import in InfluxDB

In [4]:

df_parking = pd.read_csv("../data/parking.csv", sep=",")
df_rain = pd.read_csv("../data/rain.csv", sep=",")

ts = pd.to_datetime(df_parking['published'], errors="coerce", utc=True)
df_parking['time'] = ts


df_rain['date'] = pd.to_datetime(df_rain['date'], errors="coerce")
df_rain['time'] = (
    df_rain['date']
    + pd.to_timedelta(12, unit="h")
).dt.tz_localize("Europe/Zurich", nonexistent="shift_forward", ambiguous="NaT"
).dt.tz_convert("UTC")


df_rain.head(10)

,Unnamed: 0,date,jahr,gre000d0,hto000d0,nto000d0,prestad0,rre150d0,sre000d0,tre200d0,tre200dn,tre200dx,ure200d0,time
0,0,1883-03-02,1883,NaN,NaN,50.0,996.4,0.0,NaN,2.4,0.0,6.0,NaN,1883-03-02 11:26:00+00:00
1,1,1883-03-05,1883,NaN,NaN,43.0,992.9,0.0,NaN,0.5,-4.0,7.0,NaN,1883-03-05 11:26:00+00:00
2,2,1883-03-07,1883,NaN,NaN,77.0,968.9,0.5,NaN,-0.9,-3.0,-1.0,NaN,1883-03-07 11:26:00+00:00
3,3,1883-03-14,1883,NaN,NaN,100.0,979.2,0.0,NaN,0.9,0.0,6.0,NaN,1883-03-14 11:26:00+00:00
4,4,1883-03-18,1883,NaN,NaN,70.0,976.9,0.0,NaN,4.8,-1.0,10.0,NaN,1883-03-18 11:26:00+00:00
5,5,1883-03-19,1883,NaN,NaN,50.0,972.2,0.0,NaN,6.2,-2.0,11.0,NaN,1883-03-19 11:26:00+00:00
6,6,1883-03-23,1883,NaN,NaN,63.0,980.2,0.0,NaN,-0.9,-7.0,4.0,NaN,1883-03-23 11:26:00+00:00
7,7,1883-03-24,1883,NaN,NaN,87.0,978.2,0.0,NaN,4.3,1.0,10.0,NaN,1883-03-24 11:26:00+00:00
8,8,1883-04-20,1883,NaN,NaN,100.0,980.0,10.5,NaN,9.1,8.0,NaN,NaN,1883-04-20 11:26:00+00:00
9,9,1883-04-21,1883,NaN,NaN,77.0,981.7,0.1,NaN,9.0,6.5,NaN,NaN,1883-04-21 11:26:00+00:00


### Make Connection

In [5]:
# Connection 
assert TOKEN, "Bitte Token in .env file hinzufügen"
client = InfluxDBClient(url=URL, token=TOKEN, org=ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()
health = client.health()
print("InfluxDB health:", health.status, "-", health.message)


InfluxDB health: pass - ready for queries and writes


### Daten in das Model schreiben

In [6]:
import time
from influxdb_client import Point

# ==========================================
# KONFIGURATION
# ==========================================
# Wir reduzieren die Menge pro Paket drastisch, um Timeouts zu verhindern
BATCH_SIZE = 1000  
PAUSE_SEC = 0.1    # Kurze Pause zwischen den Batches

# ==========================================
# 1. PARKDATEN
# ==========================================
print("Erstelle 'Points' für Parkdaten...")

points_parking = []
for index, row in df_parking.iterrows():
    if pd.notnull(row['time']): 
        p = Point("parking") \
            .tag("parking_id", str(row['id'])) \
            .tag("parking_name", str(row['title'])) \
            .field("free", float(row['free'])) \
            .field("total", float(row['total'])) \
            .time(row['time'])
        points_parking.append(p)

print(f"-> {len(points_parking)} Park-Punkte bereit. Starte Upload in {BATCH_SIZE}er Schritten...")

# Sicherer Batch-Loop
for i in range(0, len(points_parking), BATCH_SIZE):
    batch = points_parking[i : i + BATCH_SIZE]
    write_api.write(bucket=BUCKET, org=ORG, record=batch)
    print(f"   Park-Batch {i} bis {i + len(batch)} geschrieben.")
    time.sleep(PAUSE_SEC) # Kurze Atempause für die DB

print("Parkdaten fertig hochgeladen.\n")


# ==========================================
# 2. WETTERDATEN
# ==========================================
print("Erstelle 'Points' für Wetterdaten...")

points_rain = []
for index, row in df_rain.iterrows():
    if pd.notnull(row['time']):
        regen = float(row['rre150d0']) if pd.notnull(row['rre150d0']) else 0.0
        temp = float(row['tre200d0']) if pd.notnull(row['tre200d0']) else 0.0
        
        p = Point("weather") \
            .tag("location", "Basel") \
            .field("precipitation", regen) \
            .field("temperature", temp) \
            .time(row['time'])
        points_rain.append(p)

print(f"-> {len(points_rain)} Wetter-Punkte bereit. Starte Upload in {BATCH_SIZE}er Schritten...")

# Sicherer Batch-Loop
for i in range(0, len(points_rain), BATCH_SIZE):
    batch = points_rain[i : i + BATCH_SIZE]
    write_api.write(bucket=BUCKET, org=ORG, record=batch)
    print(f"   Wetter-Batch {i} bis {i + len(batch)} geschrieben.")
    time.sleep(PAUSE_SEC)

print("\n------------------------------------------------")
print("FERTIG! Alle Daten wurden erfolgreich importiert.")
print("------------------------------------------------")

Erstelle 'Points' für Parkdaten...
-> 901965 Park-Punkte bereit. Starte Upload in 1000er Schritten...
   Park-Batch 0 bis 1000 geschrieben.
   Park-Batch 1000 bis 2000 geschrieben.
   Park-Batch 2000 bis 3000 geschrieben.
   Park-Batch 3000 bis 4000 geschrieben.
   Park-Batch 4000 bis 5000 geschrieben.
   Park-Batch 5000 bis 6000 geschrieben.
   Park-Batch 6000 bis 7000 geschrieben.
   Park-Batch 7000 bis 8000 geschrieben.
   Park-Batch 8000 bis 9000 geschrieben.
   Park-Batch 9000 bis 10000 geschrieben.
   Park-Batch 10000 bis 11000 geschrieben.
   Park-Batch 11000 bis 12000 geschrieben.
   Park-Batch 12000 bis 13000 geschrieben.
   Park-Batch 13000 bis 14000 geschrieben.
   Park-Batch 14000 bis 15000 geschrieben.
   Park-Batch 15000 bis 16000 geschrieben.
   Park-Batch 16000 bis 17000 geschrieben.
   Park-Batch 17000 bis 18000 geschrieben.
   Park-Batch 18000 bis 19000 geschrieben.
   Park-Batch 19000 bis 20000 geschrieben.
   Park-Batch 20000 bis 21000 geschrieben.
   Park-Batch 210